In [3]:
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
input_width = 28
input_length = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
conv1_stride = 1
conv2_stride = 1
max_pool1_k = 2
max_pool2_k = 2
conv1_k = 5
conv2_k = 5
n_hidden = 1024
n_out = 10

input_hidden = (input_width//(max_pool1_k*max_pool2_k))*(input_length//(max_pool1_k*max_pool2_k))*n_conv2

In [6]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh" : tf.Variable(tf.random_normal([input_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
}

In [22]:
def conv(x,weight,bias,stride = 1):
    
    out = tf.nn.conv2d(x, weight,padding="SAME",strides= [1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out 

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [33]:
def cnn(x,weights,biases):
    x = tf.reshape(x,shape = [-1,input_length,input_width,input_channels])
    
    conv1 = conv(x,weights["wc1"],biases["bc1"],conv1_stride)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights["wc2"],biases["bc2"],conv2_stride)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_hidden])
    hidden_out_pre_act = tf.add(tf.matmul(hidden_input,weights["wh"]),biases["bh"])
    hidden_out = tf.nn.relu(hidden_out_pre_act)
    
    output = tf.add(tf.matmul(hidden_out,weights["wo"]),biases["bo"])
    
    return output

In [34]:
x = tf.placeholder("float",[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])

pred = cnn(x,weights,biases)

In [36]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [37]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [39]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_x , y:batch_y})
        total_cost += c
    print(total_cost)

970571.5295090675
33692.571149110794
20762.169775813818
14663.35279839865
12201.639997349586
12445.650433907958
11974.409281629038
11457.694037468716
9652.174350451438
8259.387688194809
7772.501088778556
5500.210237616002
6285.8617787578505
4949.887458097067
4600.96130400151
4438.641847741666
3774.216086126864
2814.4206880405545
4584.095659840253
3234.6669983992233
2846.1141978017813
2172.3249727089706
2362.1397511559726
2552.110333416612
2241.3094927449993


In [40]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:mnist.test.images,
                                              y:mnist.test.labels})
correct_predictions.sum()

9859